## 0. Install and Import Dependencies

In [8]:
!pip3 install mediapipe opencv-python pandas scikit-learn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [9]:
import mediapipe as mp #import mediapipe
import cv2 

In [10]:
mp_drawing = mp.solutions.drawing_utils #drawing helpers
mp_holistic = mp.solutions.holistic #mediapipe solutions

## 1. Make Some Detections

In [12]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image.flags.writable = False
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        #image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [22]:
results.pose_landmarks.landmark[0].visibility

0.9999454021453857

## 2. Capture Landmarks and Export to CSV

In [23]:
import csv
import os
import numpy as np

In [28]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [29]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [89]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [98]:
class_name = "YAY"

In [99]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image.flags.writable = False
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        #image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # Extract pose landmarks
            pose=results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())
            # extract face landmarks
            face=results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())
            
            #concat face and pose rows
            row = pose_row +face_row
            row.insert(0,class_name)
            
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 3. Train Custom model Using Scikit-learn

### 3.1 Read in Collected Data and Process

In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [101]:
df = pd.read_csv('coords.csv')

In [102]:
# features= information we use to predict target
#target = what we are trying to classify

X = df.drop('class', axis = 1)
y = df['class']


In [103]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 1234)

### 3.2 Train Machine Learning Classification Model

In [104]:
from sklearn.pipeline import make_pipeline #build an ML pipeline
from sklearn.preprocessing import StandardScaler #standardizes data

from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 

In [105]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [106]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [107]:
fit_models['rf'].predict(X_test)

array(['PowerStar', 'PowerStar', 'Happy', 'Sad', 'Happy', 'PowerStar',
       'YAY', 'YAY', 'YAY', 'Happy', 'ButtaBomma', 'ButtaBomma',
       'PowerStar', 'YAY', 'Happy', 'YAY', 'ButtaBomma', 'Happy',
       'PowerStar', 'ButtaBomma', 'Sad', 'Happy', 'Sad', 'ButtaBomma',
       'Happy', 'Happy', 'ButtaBomma', 'Sad', 'ButtaBomma', 'Happy',
       'PowerStar', 'PowerStar', 'Happy', 'PowerStar', 'ButtaBomma',
       'ButtaBomma', 'PowerStar', 'PowerStar', 'YAY', 'PowerStar', 'YAY',
       'YAY', 'YAY', 'Sad', 'ButtaBomma', 'Happy', 'Happy', 'Sad', 'YAY',
       'Happy', 'Happy', 'PowerStar', 'YAY', 'PowerStar', 'PowerStar',
       'PowerStar', 'YAY', 'Happy', 'Happy', 'YAY', 'ButtaBomma',
       'PowerStar', 'Sad', 'PowerStar', 'Sad', 'YAY', 'PowerStar',
       'ButtaBomma', 'YAY', 'YAY', 'ButtaBomma', 'ButtaBomma', 'Happy',
       'Sad', 'PowerStar', 'Sad', 'PowerStar', 'ButtaBomma', 'Happy',
       'ButtaBomma', 'Sad', 'YAY', 'ButtaBomma', 'YAY', 'Happy', 'Sad',
       'ButtaBomma', 'H

### 3.3 Evaluate and Serialize Model

In [108]:
from sklearn.metrics import accuracy_score # accuracy
import pickle

In [111]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test,yhat))

lr 0.9966996699669967
rc 0.9966996699669967
rf 0.9933993399339934
gb 0.9933993399339934


In [112]:
fit_models['rc'].predict(X_test)

array(['PowerStar', 'PowerStar', 'Happy', 'Sad', 'Happy', 'PowerStar',
       'YAY', 'YAY', 'YAY', 'Happy', 'ButtaBomma', 'ButtaBomma',
       'PowerStar', 'YAY', 'Happy', 'YAY', 'ButtaBomma', 'Happy',
       'PowerStar', 'ButtaBomma', 'Sad', 'Happy', 'Sad', 'ButtaBomma',
       'Happy', 'Happy', 'ButtaBomma', 'Sad', 'ButtaBomma', 'Happy',
       'PowerStar', 'PowerStar', 'Happy', 'PowerStar', 'ButtaBomma',
       'ButtaBomma', 'PowerStar', 'PowerStar', 'YAY', 'PowerStar', 'YAY',
       'YAY', 'YAY', 'Sad', 'ButtaBomma', 'Happy', 'Happy', 'Sad', 'YAY',
       'Happy', 'Happy', 'PowerStar', 'YAY', 'PowerStar', 'PowerStar',
       'PowerStar', 'YAY', 'Happy', 'Happy', 'YAY', 'ButtaBomma',
       'PowerStar', 'Sad', 'PowerStar', 'Sad', 'YAY', 'PowerStar',
       'ButtaBomma', 'YAY', 'YAY', 'ButtaBomma', 'ButtaBomma', 'Happy',
       'Sad', 'PowerStar', 'Sad', 'PowerStar', 'ButtaBomma', 'Happy',
       'ButtaBomma', 'Sad', 'YAY', 'ButtaBomma', 'YAY', 'Happy', 'Sad',
       'ButtaBomma', 'H

In [114]:
with open('body_language.pkl','wb') as f:
   pickle.dump(fit_models['rf'],f) 

## 4. Make Detections with Model

In [115]:
with open('body_language.pkl','rb') as f:
    model = pickle.load(f)

In [121]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image.flags.writable = False
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        #image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # Extract pose landmarks
            pose=results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())
            # extract face landmarks
            face=results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())
            
            #concat face and pose rows
            row = pose_row +face_row
            
            
#             row.insert(0,class_name)
            
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)

            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                                ),[640,480]).astype(int))
            cv2.rectangle(image,(coords[0],coords[1]+5), (coords[0]+len(body_language_class)*20,coords[1]-30),(245,117,16),-1)
            cv2.putText(image,body_language_class, coords,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            #Status box
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)
            cv2.putText(image, 'CLASS',(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0), 1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            cv2.putText(image, 'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0), 1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
                        
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()